# **Homework**: Data talks club data engineering zoomcamp Data loading workshop




# 1. Use a generator
- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**


In [2]:
def square_root_generator(limit):
   n = 1
   while n <= limit:
       yield n ** 0.5
       n += 1

limit = 5
generator = square_root_generator(limit)

total_sum = 0
for sqrt_value in generator:
   total_sum += sqrt_value
print(f"The total sum of the outputs of the generator with limit = 5 is {total_sum}")

The total sum of the outputs of the generator with limit = 5 is 8.382332347441762


In [4]:
limit = 13
generator = square_root_generator(limit)


for yielded, sqrt_value in enumerate(generator):
   print(f"the number {yielded + 1} yielded is {sqrt_value}")

the number 1 yielded is 1.0
the number 2 yielded is 1.4142135623730951
the number 3 yielded is 1.7320508075688772
the number 4 yielded is 2.0
the number 5 yielded is 2.23606797749979
the number 6 yielded is 2.449489742783178
the number 7 yielded is 2.6457513110645907
the number 8 yielded is 2.8284271247461903
the number 9 yielded is 3.0
the number 10 yielded is 3.1622776601683795
the number 11 yielded is 3.3166247903554
the number 12 yielded is 3.4641016151377544
the number 13 yielded is 3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [10]:
import dlt

pipeline = dlt.pipeline(destination='duckdb', dataset_name='workshop')


# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585303.581541 is LOADED and contains no failed jobs


In [11]:
# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset workshop
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585312.3916 is LOADED and contains no failed jobs


In [12]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n people table below:")

people = conn.sql("SELECT SUM(age) FROM people").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [13]:
import dlt

pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace",
                    primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585662.3809364 is LOADED and contains no failed jobs


In [14]:
# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge",
                    primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585676.0470214 is LOADED and contains no failed jobs


In [16]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n people table below:")

total_records = conn.sql("SELECT COUNT(*) FROM people")
print("Total records:")
display(total_records)

id3_age = conn.sql(" SELECT age FROM people WHERE ID = 3")
print("Age of person with ID 3 is")
display(id3_age)

people = conn.sql("SELECT SUM(age) FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:
Total records:


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            8 │
└──────────────┘

Age of person with ID 3 is


┌───────┐
│  age  │
│ int64 │
├───────┤
│    33 │
└───────┘

,sum(age)
0,266.0
